In [11]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [12]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def return_objects(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
    return objects

In [13]:
# Application Layer

def extract(bucket, objects):
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in objects], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, objects, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, objects)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [14]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2021-05-09'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    objects = return_objects(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, objects, columns, arg_date, trg_key, trg_format)

In [15]:
# run

main()

## Reading the uploaded file

In [18]:
trg_bucket = 'xetra-1234'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20210510_101629.parquet
xetra_daily_report_20210510_143346.parquet
xetra_daily_report_20210511_125520.parquet


In [20]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20210511_125520.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-05-10,9.10,9.02,8.94,9.10,145,NaN
1,AT00000FACC2,2021-05-11,8.85,8.85,8.85,8.85,300,-1.88
2,AT0000606306,2021-05-10,18.75,19.27,18.68,19.30,3077,NaN
3,AT0000606306,2021-05-11,19.11,19.11,19.11,19.11,1200,-0.83
4,AT0000609607,2021-05-10,16.48,17.46,16.48,17.46,331,NaN
...,...,...,...,...,...,...,...,...
5598,XS2265369731,2021-05-11,10.28,10.28,10.28,10.28,0,-1.63
5599,XS2265370234,2021-05-10,24.23,24.59,24.23,24.59,300,NaN
5600,XS2265370234,2021-05-11,24.42,24.42,24.42,24.42,0,-0.69
5601,XS2284324667,2021-05-10,26.87,26.63,26.57,26.90,8302,NaN


In [2]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

arg_date = '2021-05-09'
src_format = '%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = 'xetra-1234'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
trg_key = 'xetra_daily_report_'
trg_format = '.parquet'

# Init
s3 = boto3.resource('s3')
bucket_src = s3.Bucket(src_bucket)
bucket_trg = s3.Bucket(trg_bucket)
    

In [41]:
objects = bucket_src.objects.filter(Prefix='2021-05-11')
for obj in objects:
    print(obj)

s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR00.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR01.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR02.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR03.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR04.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR05.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR06.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR07.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11/2021-05-11_BINS_XETR08.csv')
s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-11

In [42]:
obj = bucket_src.Object('2021-05-11/2021-05-11_BINS_XETR10.csv').get().get('Body').read().decode('utf-8')
data = StringIO(obj)
df = pd.read_csv(data)
df

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000ETFL060,EL4F,DK DAX (AUSSCHUETTEND),ETF,EUR,2504259,2021-05-11,10:00,64.980,64.980,64.980,64.980,1763,1
1,DE0005933931,EXS1,ISHS CORE DAX UC.ETF EOA,ETF,EUR,2504265,2021-05-11,10:00,128.000,128.020,128.000,128.020,1083,3
2,LU0274211480,DBXD,XTR.DAX 1C,ETF,EUR,2504269,2021-05-11,10:00,144.380,144.380,144.360,144.360,265,2
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2021-05-11,10:00,206.300,206.300,206.300,206.300,3,1
4,DE000A0F5UF5,EXXT,ISHARES NASDAQ-100 U.ETF,ETF,EUR,2504303,2021-05-11,10:00,105.700,105.700,105.700,105.700,230,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11319,DE000A2LQ884,AG1,AUTO1 GROUP SE INH O.N.,Common stock,EUR,6080964,2021-05-11,10:59,43.670,43.740,43.660,43.740,301,3
11320,DE000A3H3LL2,VTWR,VANTAGE TOWERS AG NA O.N.,Common stock,EUR,6204589,2021-05-11,10:59,26.220,26.220,26.220,26.220,144,2
11321,IE00BMDH1538,HDR0,VANECK HYD.ECO U.ETF ADLA,ETF,EUR,6270784,2021-05-11,10:59,14.062,14.062,14.062,14.062,30,1
11322,DE000A3GPSP7,VETH,VANECK ETHEREUM ETN29,ETN,EUR,6270785,2021-05-11,10:59,20.300,20.300,20.300,20.300,25,1


In [20]:
def return_objects(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
    return objects

In [23]:
ob = return_objects(bucket_src, '2021-09-01', '%Y-%m-%d')

In [24]:
def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

In [25]:
df = pd.concat([read_csv_to_df(bucket_src, obj) for obj in ob], ignore_index=True)

KeyboardInterrupt: 

In [19]:
csv_obj = bucket_src.Object('2019-05-19/2019-05-19_BINS_XETR17.csv').get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj)
df = pd.read_csv(data)
df

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
